In [1]:
from pathlib import Path
from rich import print

from langchain.memory import ConversationBufferMemory
from langchain.utilities import SQLDatabase
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableConfig

In [2]:
llm = ChatOllama(model='llama2')

In [3]:
db_path = Path.cwd() / 'pantry.db'
rel = db_path.relative_to(Path.cwd())
db_string = f"sqlite:///{rel}"
db_string

'sqlite:///pantry.db'

In [4]:
db = SQLDatabase.from_uri(db_string)

In [5]:
print(db.get_table_info())

CREATE TABLE pantry (
        id INTEGER, 
        name TEXT NOT NULL, 
        in_stock BOOLEAN, 
        PRIMARY KEY (id)
)

/*
3 rows from pantry table:
id      name    in_stock
1       Quinoa  True
2       Apples  False
3       Oranges True
*/

In [6]:
db.run("SELECT name FROM pantry WHERE in_stock == TRUE")

"[('Quinoa',), ('Oranges',), ('Bananas',)]"

In [7]:
def get_schema(_):
    return db.get_table_info()


def run_query(query):
    return db.run(query)

In [8]:
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{sql_schema}

Question: {question}
SQL Query: """

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Given an input question, convert it to a SQL query. No pre-amble."),
        MessagesPlaceholder(variable_name="history"),
        ("human", template),
    ]
)

In [10]:
sql_chain = (
    RunnablePassthrough.assign(
        schema=get_schema,
    )
    | prompt
    | llm.bind(stop=["\nSQLResult: "])
    | StrOutputParser()
)

In [11]:
memory = ConversationBufferMemory(return_messages=True)

In [12]:
sql_chain = (
    RunnablePassthrough.assign(
        sql_schema=get_schema,
        history=RunnableLambda(lambda x: memory.load_memory_variables(x)["history"]),
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
def save(input_output):
    output = {"output": input_output.pop("output")}
    memory.save_context(input_output, output)
    return output["output"]

In [14]:
sql_response_memory = RunnablePassthrough.assign(output=sql_chain) | save

In [15]:
sql_response_memory.invoke({"question":"What food ingredients are in stock?"})

'SELECT name FROM pantry WHERE in_stock = True;'

In [16]:
# Chain to answer
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""  # noqa: E501
prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question and SQL response, convert it to a natural "
            "language answer. No pre-amble.",
        ),
        ("human", template),
    ]
)

In [17]:
# Supply the input types to the prompt
class InputType(BaseModel):
    question: str

In [18]:
chain = (
    RunnablePassthrough.assign(query=sql_response_memory).with_types(
        input_type=InputType
    )
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | llm
    | StrOutputParser()
)

In [20]:
chain.invoke({'question':"What food items are not in stock?"})

'According to the database, Apples and Pasta are not currently in stock.'